In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [2]:
np.random.seed(1)

In [3]:
df = pd.read_csv('school.csv',sep= ',')
df.head()

,CaseID$,AID,ELYEAR1,ELYEAR2,ELYEAR3,ELYEAR4,ELYEAR5,ELYEAR6,ELMAT945,ELGLV945,...,EASFIX5,EASFIX6,EASFIXC,EAOFIX1,EAOFIX2,EAOFIX3,EAOFIX4,EAOFIX5,EAOFIX6,EAOFIXC
0,1,57100270,1991,1992,1993,1994,9992,9992,4,10.0,...,9992.0,9992.0,0.667,0.000,0.357,0.750,0.444,9992.0,9992.0,0.383
1,2,57103869,1991,1992,1993,1994,1995,9992,4,12.0,...,9994.0,9992.0,0.667,0.000,0.500,0.571,1.000,1.0,9992.0,0.567
2,3,57104676,1996,1997,1998,1999,9992,9992,9993,9993.0,...,9992.0,9992.0,0.000,0.071,0.000,0.250,0.000,9992.0,9992.0,0.075
3,4,57111071,1995,1996,1997,1998,9992,9992,9993,9993.0,...,9992.0,9992.0,0.000,0.000,0.000,0.000,0.000,9992.0,9992.0,0.000
4,5,57111786,1994,1995,1996,1997,9992,9992,1,9.0,...,9992.0,9992.0,0.000,0.000,0.000,0.000,0.000,9992.0,9992.0,0.000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3947 entries, 0 to 3946
Data columns (total 81 columns):
CaseID$     3947 non-null int64
AID         3947 non-null int64
ELYEAR1     3947 non-null int64
ELYEAR2     3947 non-null int64
ELYEAR3     3947 non-null int64
ELYEAR4     3947 non-null int64
ELYEAR5     3947 non-null int64
ELYEAR6     3947 non-null int64
ELMAT945    3947 non-null int64
ELGLV945    3947 non-null float64
ELY1NINE    3947 non-null int64
EAMSQ1      3947 non-null int64
EAMSQ2      3947 non-null int64
EAMSQ3      3947 non-null int64
EAMSQ4      3947 non-null int64
EAMSQ5      3947 non-null int64
EAMSQ6      3947 non-null int64
EAMSQH      3947 non-null int64
EAMSQB1     3947 non-null int64
EAMSQB2     3947 non-null int64
EAMSQB3     3947 non-null int64
EAMSQB4     3947 non-null int64
EAMSQB5     3947 non-null int64
EAMSQB6     3947 non-null int64
EAMSQBH     3947 non-null int64
EASSQ1      3947 non-null int64
EASSQ2      3947 non-null int64
EASSQ3      3947 non-null i

## Data Cleaning

In [5]:
target = 'ELY1NINE'

### Replace empty or errors by np.nan

In [6]:
# replace empty/erros by np.nan
df = df.replace(r'^\s+$', np.nan, regex=True)
df = df.replace('\t','',regex=True)
df = df.replace(' ','',regex=True)
df = df.replace('\?',np.nan,regex=True)
df = df.replace('\<',np.nan,regex=True)
#df

### Remove outliers

In [7]:
# Drop the column `Code` as it is unrelated to the dependent variable
df = df.drop(['AID','CaseID$'],axis=1)
df.head()

,ELYEAR1,ELYEAR2,ELYEAR3,ELYEAR4,ELYEAR5,ELYEAR6,ELMAT945,ELGLV945,ELY1NINE,EAMSQ1,...,EASFIX5,EASFIX6,EASFIXC,EAOFIX1,EAOFIX2,EAOFIX3,EAOFIX4,EAOFIX5,EAOFIX6,EAOFIXC
0,1991,1992,1993,1994,9992,9992,4,10.0,1,4,...,9992.0,9992.0,0.667,0.000,0.357,0.750,0.444,9992.0,9992.0,0.383
1,1991,1992,1993,1994,1995,9992,4,12.0,1,2,...,9994.0,9992.0,0.667,0.000,0.500,0.571,1.000,1.0,9992.0,0.567
2,1996,1997,1998,1999,9992,9992,9993,9993.0,1,4,...,9992.0,9992.0,0.000,0.071,0.000,0.250,0.000,9992.0,9992.0,0.075
3,1995,1996,1997,1998,9992,9992,9993,9993.0,1,4,...,9992.0,9992.0,0.000,0.000,0.000,0.000,0.000,9992.0,9992.0,0.000
4,1994,1995,1996,1997,9992,9992,1,9.0,1,4,...,9992.0,9992.0,0.000,0.000,0.000,0.000,0.000,9992.0,9992.0,0.000


### Remove bad columns

In [8]:
def remove_bad_columns(df,bad_column_threshold):
    # find bad columns having too many missing values
    n_null = np.array(df.isnull().sum(axis=0))
    bad_col = np.array([]).astype(int)
    for i in range(len(n_null)):
        if n_null[i] >= bad_column_threshold:
            bad_col = np.append(bad_col,i)

    #print(bad_col)
    print('number of bad columns:',len(bad_col))

    # delete bad columns
    df = df.drop(df.columns[bad_col],axis=1)
    #df.info()
    return df  

In [9]:
df = remove_bad_columns(df,100)
df.info()

number of bad columns: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3947 entries, 0 to 3946
Data columns (total 79 columns):
ELYEAR1     3947 non-null int64
ELYEAR2     3947 non-null int64
ELYEAR3     3947 non-null int64
ELYEAR4     3947 non-null int64
ELYEAR5     3947 non-null int64
ELYEAR6     3947 non-null int64
ELMAT945    3947 non-null int64
ELGLV945    3947 non-null float64
ELY1NINE    3947 non-null int64
EAMSQ1      3947 non-null int64
EAMSQ2      3947 non-null int64
EAMSQ3      3947 non-null int64
EAMSQ4      3947 non-null int64
EAMSQ5      3947 non-null int64
EAMSQ6      3947 non-null int64
EAMSQH      3947 non-null int64
EAMSQB1     3947 non-null int64
EAMSQB2     3947 non-null int64
EAMSQB3     3947 non-null int64
EAMSQB4     3947 non-null int64
EAMSQB5     3947 non-null int64
EAMSQB6     3947 non-null int64
EAMSQBH     3947 non-null int64
EASSQ1      3947 non-null int64
EASSQ2      3947 non-null int64
EASSQ3      3947 non-null int64
EASSQ4      3947 non-null int64
EA

### Remove bad rows

In [10]:
 # Find rows where target is missing
def find_missing_target_rows(df,target):
    # find rows where target is missing
    missing_row = df[target].isnull()
    print('Number of rows where target are missing:')
    print(sum(missing_row))

    #df = df[~missing_row]
    missing_row_indices = np.array([t for t in range(df.shape[0]) if missing_row[t]])
    
    return missing_row_indices

In [11]:
missing_target_rows = find_missing_target_rows(df,target)

Number of rows where target are missing:
0


We find bad rows which contain too many missing values, then remove them.

In [12]:
def find_bad_rows(df,bad_row_threshold):   
    # find bad rows having too many missing values
    n_null = np.array(df.isnull().sum(axis=1))
    bad_row = np.array([]).astype(int)
    for t in range(len(n_null)):
        if n_null[t] >= bad_row_threshold:
            bad_row = np.append(bad_row,t)

    #print(bad_row)
    print('number of bad rows:',len(bad_row))

    # delete bad rows
    #df = df.drop(bad_row)
    #df.info()
    return bad_row

In [13]:
bad_rows = find_bad_rows(df,5)
df.info()

number of bad rows: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3947 entries, 0 to 3946
Data columns (total 79 columns):
ELYEAR1     3947 non-null int64
ELYEAR2     3947 non-null int64
ELYEAR3     3947 non-null int64
ELYEAR4     3947 non-null int64
ELYEAR5     3947 non-null int64
ELYEAR6     3947 non-null int64
ELMAT945    3947 non-null int64
ELGLV945    3947 non-null float64
ELY1NINE    3947 non-null int64
EAMSQ1      3947 non-null int64
EAMSQ2      3947 non-null int64
EAMSQ3      3947 non-null int64
EAMSQ4      3947 non-null int64
EAMSQ5      3947 non-null int64
EAMSQ6      3947 non-null int64
EAMSQH      3947 non-null int64
EAMSQB1     3947 non-null int64
EAMSQB2     3947 non-null int64
EAMSQB3     3947 non-null int64
EAMSQB4     3947 non-null int64
EAMSQB5     3947 non-null int64
EAMSQB6     3947 non-null int64
EAMSQBH     3947 non-null int64
EASSQ1      3947 non-null int64
EASSQ2      3947 non-null int64
EASSQ3      3947 non-null int64
EASSQ4      3947 non-null int64
EASSQ

In [14]:
del_rows = np.union1d(missing_target_rows,bad_rows)
print('number of rows need to delete:',len(del_rows))

df = df.drop(del_rows)

number of rows need to delete: 0


### Separate target and attributes

In [15]:
dfx = df.drop(target,axis=1)
dfy = df[target]

### Find variable type

In [16]:
# number of uniques of each column (excluding NaN)
nu = np.array([len(pd.unique(dfx[col].dropna())) for col in dfx.columns])
print('number of uniques of each variable:')
print(nu)

number of uniques of each variable:
[  13   12   11   11    9    8    8   25   11   12   12   12   12   11
   10   11   12   12   12   12   11   11    8    9    9    9    9    8
    7    8    9    9    9    9    8    8   40   34   36   36   20   12
  183   27   32   33   31   20   14  159  366  355  360  307   92   27
 1726   15   13   14   14    9    9   52   10   11   13   12    9    9
   45   95   90   97   86   34   14  405]


In [17]:
def define_variable_type(df,nu):
    i_binary = [] ; i_category = [] ; i_continuous = []
    for i in range(len(nu)):
        if nu[i] == 2: # binary 
            i_binary.append(i)
        elif nu[i] < 5: # !!!! NOTE: this is not always correct, depending on data
            i_category.append(i)
        else:
            i_continuous.append(i)

    print('i_binary:',i_binary)
    print('i_category:',i_category)   
    #i_binary, i_category, i_continuous
    
    variable_type  = np.ones(len(nu))  # binary
    variable_type[i_category] = 2   # categorical
    variable_type[i_continuous] = 3 # continuous

    return variable_type #,i_binary,i_category,i_continuous

In [18]:
variable_type = define_variable_type(dfx,nu)
print('variable type:',variable_type)

i_binary: []
i_category: []
variable type: [3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3.]


### Impute missing values of attributes

In [19]:
def impute_missing(df,variable_type):
    # impute binary and categorical variables by the most frequency (in each column)
    # continuous variable by median
    # input: df: pandas data frame, variable_type: list
    # output: df2: pandas data frame
    df2 = df.copy()
    for i,col in enumerate(df.columns):
        if variable_type[i] < 3: # binary or caterogy
            df2[col] = df[col].fillna(df[col].mode().iloc[0])
        else: # continuous
            df2[col] = df[col].fillna(df[col].median())    
    return df2       

In [20]:
dfx_imputed = impute_missing(dfx,variable_type)
dfx_imputed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3947 entries, 0 to 3946
Data columns (total 78 columns):
ELYEAR1     3947 non-null int64
ELYEAR2     3947 non-null int64
ELYEAR3     3947 non-null int64
ELYEAR4     3947 non-null int64
ELYEAR5     3947 non-null int64
ELYEAR6     3947 non-null int64
ELMAT945    3947 non-null int64
ELGLV945    3947 non-null float64
EAMSQ1      3947 non-null int64
EAMSQ2      3947 non-null int64
EAMSQ3      3947 non-null int64
EAMSQ4      3947 non-null int64
EAMSQ5      3947 non-null int64
EAMSQ6      3947 non-null int64
EAMSQH      3947 non-null int64
EAMSQB1     3947 non-null int64
EAMSQB2     3947 non-null int64
EAMSQB3     3947 non-null int64
EAMSQB4     3947 non-null int64
EAMSQB5     3947 non-null int64
EAMSQB6     3947 non-null int64
EAMSQBH     3947 non-null int64
EASSQ1      3947 non-null int64
EASSQ2      3947 non-null int64
EASSQ3      3947 non-null int64
EASSQ4      3947 non-null int64
EASSQ5      3947 non-null int64
EASSQ6      3947 non-null i

## Data Processing

### Attributes

In [21]:
def convert_binary_and_category(x,variable_type):
    """
    convert binary to +-1, category to one hot; remain continuous.
    """    
    onehot_encoder = OneHotEncoder(sparse=False,categories='auto')

    # create 2 initial columns
    x_new = np.zeros((x.shape[0],2))

    for i,i_type in enumerate(variable_type):
        if i_type == 1: # binary
            unique_value = np.unique(x[:,i])
            x1 = np.array([-1. if value == unique_value[0] else 1. for value in x[:,i]])        
            x_new = np.hstack((x_new,x1[:,np.newaxis]))

        elif i_type == 2: # category
            x1 = onehot_encoder.fit_transform(x[:,i].reshape(-1,1))
            x_new = np.hstack((x_new,x1))
            
        else: # continuous      
            x_new = np.hstack((x_new,x[:,i][:,np.newaxis]))      

    # drop the 2 initial column
    x_new = x_new[:,2:]
    
    return x_new.astype(float)

In [22]:
# convert x
x = np.array(dfx_imputed)
x_new = convert_binary_and_category(x,variable_type)

print(x_new.shape)
print(x_new)

(3947, 78)
[[1.991e+03 1.992e+03 1.993e+03 ... 9.992e+03 9.992e+03 3.830e-01]
 [1.991e+03 1.992e+03 1.993e+03 ... 1.000e+00 9.992e+03 5.670e-01]
 [1.996e+03 1.997e+03 1.998e+03 ... 9.992e+03 9.992e+03 7.500e-02]
 ...
 [1.996e+03 1.997e+03 1.998e+03 ... 9.992e+03 9.992e+03 0.000e+00]
 [1.995e+03 1.996e+03 1.997e+03 ... 9.992e+03 9.992e+03 0.000e+00]
 [1.995e+03 1.996e+03 1.997e+03 ... 9.992e+03 9.992e+03 0.000e+00]]


### Target

In [23]:
y = np.array(dfy)
#print(np.unique(y,return_counts=True))

# convert taget to 0 and 1
y_new = y
#y_new = np.ones(y.shape[0])
#y_new[y =='No'] = 0

print(np.unique(y_new,return_counts=True))

(array([0, 1]), array([  68, 3879]))


In [24]:
# combine X and y and save to a file
xy_new = np.hstack((x_new,y_new[:,np.newaxis]))
np.savetxt('school_processed.dat',xy_new,fmt='%f')